In [52]:
# Import necessary libraries
import os
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
import numpy as np

# Set working directory
#os.chdir("C:/Users/Hp/Desktop/HCES2223/Data")

######Analysis###

In [54]:
Level1=pd.read_parquet(r'C:\Users\DIID\Desktop\HCES2223\CSV\hces22_lvl_01.TXT.parquet')
Level2=pd.read_parquet(r'C:\Users\DIID\Desktop\HCES2223\CSV\hces22_lvl_02.TXT.parquet')
Level3=pd.read_parquet(r'C:\Users\DIID\Desktop\HCES2223\CSV\hces22_lvl_03.TXT.parquet')
Level11=pd.read_parquet(r'C:\Users\DIID\Desktop\HCES2223\CSV\hces22_lvl_11.TXT.parquet')
Level14=pd.read_parquet(r'C:\Users\DIID\Desktop\HCES2223\CSV\hces22_lvl_14.TXT.parquet')
Level15=pd.read_parquet(r'C:\Users\DIID\Desktop\HCES2223\CSV\hces22_lvl_15.TXT.parquet')


In [56]:
Level2=Level2[Level2.columns[16:]]
Level3=Level3[Level3.columns[16:]]
Level11=Level11[Level11.columns[16:]]
Level14=Level14[Level14.columns[16:]]
Level15=Level15[Level15.columns[16:]]

In [58]:
Level11.columns

Index(['Questionnaire No.', 'Level',
       'Which online item purchased/Paid during last 365 days - Clothing',
       'Which online item purchased/Paid during last 365 days- Footwear',
       'Which online item purchased/Paid during last 365 days - Furniture & fixtures',
       'Which online item purchased/Paid during last 365 days - Mobile handset',
       'Which online item purchased/Paid during last 365 days- Personal goods (laptop/PC, tablet, clock, watch, spectacles, contact \r\r\nlenses, etc.)',
       'Which online item purchased/Paid during last 365 days- Goods for recreation (TV, camera, pen-drive, musical instruments, \r\r\netc.) \r\r\n',
       'Which online item purchased/Paid during last 365 days -Cooking & other household appliances',
       'Which online item purchased/Paid during last 365 days - Crockery & utensils',
       'Which online item purchased/Paid during last 365 days - Sports goods',
       'Which online item purchased/Paid during last 365 days- Medical equi

In [60]:
Level11=Level11[['HH_ID','Which online item purchased/Paid during last 365 days - Clothing',
       'Which online item purchased/Paid during last 365 days- Footwear',
       'Which online item purchased/Paid during last 365 days - Furniture & fixtures',
       'Which online item purchased/Paid during last 365 days - Mobile handset',
       'Which online item purchased/Paid during last 365 days- Personal goods (laptop/PC, tablet, clock, watch, spectacles, contact \r\r\nlenses, etc.)',
       'Which online item purchased/Paid during last 365 days- Goods for recreation (TV, camera, pen-drive, musical instruments, \r\r\netc.) \r\r\n',
       'Which online item purchased/Paid during last 365 days -Cooking & other household appliances',
       'Which online item purchased/Paid during last 365 days - Crockery & utensils',
       'Which online item purchased/Paid during last 365 days - Sports goods',
       'Which online item purchased/Paid during last 365 days- Medical equipment',
       'Which online item purchased/Paid during last 365 days- Bedding',
                 'Whether household possessed one or more item as on the date of the survey- Television',
       'Whether household possessed one or more item as on the date of the survey- Radio',
       'Whether household possessed one or more item as on the date of the survey - Laptop/PC',
       'Whether household possessed one or more item as on the date of the survey- Mobile handset',
       'Whether household possessed one or more item as on the date of the survey- Bicycle',
       'Whether household possessed one or more item as on the date of the survey- Motorcycle, scooter',
       'Whether household possessed one or more item as on the date of the survey- Motor car/jeep/van',
       'Whether household possessed one or more item as on the date of the survey- Trucks',
       'Whether household possessed one or more item as on the date of the survey - Animal cart',
       'Whether household possessed one or more item as on the date of the survey- Refrigerator',
       'Whether household possessed one or more item as on the date of the survey- Washing machine',
       'Whether household possessed one or more item as on the date of the survey- Air conditioner/air cooler']]

In [62]:
Level1=Level1[['HH_ID','Sector','State','NSS-Region','District','Multiplier']]
Level3=Level3[['HH_ID','Household Type', 'Religion of the head of the household','Social Group of the head of the household', 
               'HH Size (For FDQ)','NCO-2015 Code(3-digit)', 'NIC-2008 Code(5-digit)']]
Level1_3=Level1.merge(Level3, on ='HH_ID', how= 'left')
Level1_311=Level1_3.merge(Level11, on ='HH_ID', how= 'left')

In [64]:
# Extract data in a single step for each household size category
HH_Visit= Level15[Level15['Questionnaire No.'].isin(['F', 'C', 'D'])]
# Pivot the household sizes into separate columns for each questionnaire category
HH_Visit = HH_Visit.pivot(index='HH_ID', columns='Questionnaire No.', values='Household size').reset_index()

In [66]:
HH_Data= Level1_311.merge(HH_Visit, on ='HH_ID', how= 'left')

In [76]:
def expense_summariser(df, item_list, multiplier, column_name_to_be_taken):
    test= df[df['Item_Code'].isin(item_list)]
    test= test.groupby('HH_ID')['Value_in_Rs'].sum() * multiplier
    test=test.reset_index(name= column_name_to_be_taken)
    df_HHID= Level14[['HH_ID']].drop_duplicates()
    test= df_HHID.merge(test, on ='HH_ID', how= 'left')
    return test

In [78]:
#Food Items
# Subset data for level 14
L14subset= Level14[['HH_ID', 'Questionnaire No.', 'Item Code', 'Value (in Rs)']]
L14subset.columns = ['HH_ID', 'Questionnaire_No', 'Item_Code', 'Value_in_Rs']
L14subset['Value_in_Rs']=L14subset['Value_in_Rs'].astype(float)
L14subset['Item_Code']=L14subset['Item_Code'].astype(int)
FoodSummary = L14subset[L14subset['Questionnaire_No'] == "F"]

# Food expenses
Food30 = expense_summariser(FoodSummary, [129, 139, 159, 179],1, "Food_30days")  # 30-day items
Food07 = expense_summariser(FoodSummary, [169, 219, 239, 249, 199, 189, 269, 279, 289, 299], 30 / 7, "Food_07days")  # 7-day items

# Total Food expenses in 30 days
Food = Food30.merge(Food07 , on ='HH_ID', how= 'left')
Food=Food.fillna(0)
Food['Food_Expense']= Food['Food_30days'] + Food['Food_07days']

C:\Users\DIID\AppData\Local\Temp\ipykernel_21552\3137296924.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L14subset['Value_in_Rs']=L14subset['Value_in_Rs'].astype(float)
C:\Users\DIID\AppData\Local\Temp\ipykernel_21552\3137296924.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L14subset['Item_Code']=L14subset['Item_Code'].astype(int)


In [80]:
# Separate data into categories: Consumables
ConsumablesSummary = L14subset[L14subset['Questionnaire_No'] == "C"]
# Consumables expenses $ 539 for imputed
Consumables30 = expense_summariser(ConsumablesSummary, [349, 459, 479, 429, 519, 499, 439, 529],1,"Consumable_30days")  # 30-day items
Consumables07 = expense_summariser(ConsumablesSummary, [309, 319, 329], 30 / 7,"Consumable_07days")  # 7-day items
Consumables365 = expense_summariser(ConsumablesSummary, [409, 419, 899], 30 / 365,"Consumable_365days")  # 365-day items

Consumable = Consumables30.merge(Consumables07 , on ='HH_ID', how= 'left')
Consumable=Consumable.merge(Consumables365 , on ='HH_ID', how= 'left')
Consumable=Consumable.fillna(0)
Consumable['Consumable_Expense']= Consumable['Consumable_30days'] + Consumable['Consumable_07days'] + Consumable['Consumable_365days']

In [82]:
# Durables
DurablesSummary = L14subset[L14subset['Questionnaire_No'] == "D"]

# Durables expenses
Durables365 = expense_summariser(DurablesSummary, DurablesSummary['Item_Code'].tolist(), 30 / 365,"Durable_365days")  # 365-day items
Durables365=Durables365.fillna(0)

In [84]:
Consumption_All= Food.merge(Consumable, on ='HH_ID', how= 'left')
Consumption_All= Consumption_All.merge(Durables365, on ='HH_ID', how= 'left')

In [86]:
Consumption_All= Food.merge(Consumable, on ='HH_ID', how= 'left')
Consumption_All= Consumption_All.merge(Durables365, on ='HH_ID', how= 'left')
HH_Data=HH_Data.merge(Consumption_All[['HH_ID','Food_Expense','Consumable_Expense','Durable_365days']], on= 'HH_ID', how= 'left')
HH_Data[['C', 'D', 'F']]=HH_Data[['C', 'D', 'F']].astype(int)

In [88]:
HH_Data['Multiplier']=HH_Data['Multiplier'].astype(float)
HH_Data['Weight']=(HH_Data['Multiplier'])/100
HH_Data['TotalExpense'] = HH_Data['Food_Expense'] + (HH_Data['Consumable_Expense'])*(HH_Data['F'])/(HH_Data['C']) + (HH_Data['Durable_365days']) * (HH_Data['F']) /(HH_Data['D'])
HH_Data['MPCE'] = HH_Data['TotalExpense'] / HH_Data['F']
HH_Data['WtdTotalExpense']=HH_Data['TotalExpense']*HH_Data['Weight']
HH_Data['WtdHH_in_F']=HH_Data['F']*HH_Data['Weight']

In [90]:
(HH_Data.groupby(['Sector'])['WtdTotalExpense'].sum() / HH_Data.groupby(['Sector'])['WtdHH_in_F'].sum()).reset_index(name='MPCE')

,Sector,MPCE
0,1,3773.052260
1,2,6458.691602


In [92]:
(HH_Data.groupby(['State','Sector'])['WtdTotalExpense'].sum() / HH_Data.groupby(['State','Sector'])['WtdHH_in_F'].sum()).reset_index(name='MPCE')

,State,Sector,MPCE
0,01,1,4295.693203
1,01,2,6178.501773
2,02,1,5560.850141
3,02,2,8075.279338
4,03,1,5314.744092
...,...,...,...
67,35,2,10268.150702
68,36,1,4802.229901
69,36,2,8158.441165
70,37,1,4035.291713


In [126]:
HH_Level_Data=HH_Data[['HH_ID', 'Sector', 'State', 'NSS-Region', 'District',
       'Household Type', 'Religion of the head of the household',
       'Social Group of the head of the household', 'HH Size (For FDQ)',
       'NCO-2015 Code(3-digit)', 'NIC-2008 Code(5-digit)',
       'Which online item purchased/Paid during last 365 days - Clothing',
       'Which online item purchased/Paid during last 365 days- Footwear',
       'Which online item purchased/Paid during last 365 days - Furniture & fixtures',
       'Which online item purchased/Paid during last 365 days - Mobile handset',
       'Which online item purchased/Paid during last 365 days- Personal goods (laptop/PC, tablet, clock, watch, spectacles, contact \r\r\nlenses, etc.)',
       'Which online item purchased/Paid during last 365 days- Goods for recreation (TV, camera, pen-drive, musical instruments, \r\r\netc.) \r\r\n',
       'Which online item purchased/Paid during last 365 days -Cooking & other household appliances',
       'Which online item purchased/Paid during last 365 days - Crockery & utensils',
       'Which online item purchased/Paid during last 365 days - Sports goods',
       'Which online item purchased/Paid during last 365 days- Medical equipment',
       'Which online item purchased/Paid during last 365 days- Bedding',
       'Whether household possessed one or more item as on the date of the survey- Television',
       'Whether household possessed one or more item as on the date of the survey- Radio',
       'Whether household possessed one or more item as on the date of the survey - Laptop/PC',
       'Whether household possessed one or more item as on the date of the survey- Mobile handset',
       'Whether household possessed one or more item as on the date of the survey- Bicycle',
       'Whether household possessed one or more item as on the date of the survey- Motorcycle, scooter',
       'Whether household possessed one or more item as on the date of the survey- Motor car/jeep/van',
       'Whether household possessed one or more item as on the date of the survey- Trucks',
       'Whether household possessed one or more item as on the date of the survey - Animal cart',
       'Whether household possessed one or more item as on the date of the survey- Refrigerator',
       'Whether household possessed one or more item as on the date of the survey- Washing machine',
       'Whether household possessed one or more item as on the date of the survey- Air conditioner/air cooler','TotalExpense']]

In [132]:
HH_Level_Data.columns

Index(['HH_ID', 'Sector', 'State', 'NSS-Region', 'District', 'Household Type',
       'Religion of the head of the household',
       'Social Group of the head of the household', 'HH Size (For FDQ)',
       'NCO-2015 Code(3-digit)', 'NIC-2008 Code(5-digit)',
       'Which online item purchased/Paid during last 365 days - Clothing',
       'Which online item purchased/Paid during last 365 days- Footwear',
       'Which online item purchased/Paid during last 365 days - Furniture & fixtures',
       'Which online item purchased/Paid during last 365 days - Mobile handset',
       'Which online item purchased/Paid during last 365 days- Personal goods (laptop/PC, tablet, clock, watch, spectacles, contact \r\r\nlenses, etc.)',
       'Which online item purchased/Paid during last 365 days- Goods for recreation (TV, camera, pen-drive, musical instruments, \r\r\netc.) \r\r\n',
       'Which online item purchased/Paid during last 365 days -Cooking & other household appliances',
       'Which onli

In [136]:
HH_Level_Data.columns=['HH_ID', 'Sector', 'State', 'NSS-Region', 'District','Household Type', 'Religion of the head of the household',
                       'Social Group of the head of the household', 'HH Size (For FDQ)','NCO_3D', 'NIC_5D',
                       'Is_online_Clothing_Purchased_Last365',
                       'Is_online_Footwear_Purchased_Last365',
                       'Is_online_Furniture_fixturesPurchased_Last365',
                       'Is_online_Mobile_Handset_Purchased_Last365' ,
                       'Is_online_Personal_Goods_Purchased_Last365' ,
                       'Is_online_Recreation_Goods_Purchased_Last365' ,
                       'Is_online_Household_Appliances_Purchased_Last365',
                      'Is_online_Crockery_Utensils_Purchased_Last365',
                      'Is_online_Sports_Goods_Purchased_Last365',
                     'Is_online_Medical_Equipment_Purchased_Last365',
                     'Is_online_Bedding_Purchased_Last365',
                       'Is_HH_Have_Television', 'Is_HH_Have_Radio', 'Is_HH_Have_Laptop_PC',
                       'Is_HH_Have_Mobile_handset', 'Is_HH_Have_Bicycle',
                       'Is_HH_Have_Motorcycle_scooter', 'Is_HH_Have_Motorcar_jeep_van',
                       'Is_HH_Have_Trucks', 'Is_HH_Have_Animal_cart',
                       'Is_HH_Have_Refrigerator', 'Is_HH_Have_Washing_machine',
                       'Is_HH_Have_Airconditioner_aircooler','TotalExpense']

In [144]:
HH_Level_Data

,HH_ID,Sector,State,NSS-Region,District,Household Type,Religion of the head of the household,Social Group of the head of the household,HH Size (For FDQ),NCO_3D,...,Is_HH_Have_Mobile_handset,Is_HH_Have_Bicycle,Is_HH_Have_Motorcycle_scooter,Is_HH_Have_Motorcar_jeep_van,Is_HH_Have_Trucks,Is_HH_Have_Animal_cart,Is_HH_Have_Refrigerator,Is_HH_Have_Washing_machine,Is_HH_Have_Airconditioner_aircooler,TotalExpense
0,HCES2022655561010131113011101202201,1,01,013,11,2,2,9,5,332,...,1,1,1,1,,,1,1,,30903.434442
1,HCES2022655561010131113011101202301,1,01,013,11,6,2,9,6,931,...,1,1,1,,,,1,1,,24208.706458
2,HCES2022655561010131113011101202302,1,01,013,11,2,2,9,8,833,...,1,1,1,,,,1,1,,31282.904110
3,HCES2022655561010131113011101202303,1,01,013,11,2,2,9,4,142,...,1,1,,1,,,1,1,,25315.976517
4,HCES2022655561010131113011101202304,1,01,013,11,2,2,9,4,833,...,1,1,,,,,1,1,,17065.553816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261741,HCES20223951022828313220210228101314,2,28,283,13,1,1,3,2,832,...,1,,1,1,,,1,1,,25161.694716
261742,HCES20223951022828313220210228101315,2,28,283,13,2,1,9,4,216,...,1,,1,1,,,1,1,1,32385.309198
261743,HCES20223951022828313220210228101316,2,28,283,13,2,1,9,3,411,...,1,,1,,,,1,1,,28774.367906
261744,HCES20223951022828313220210228101317,2,28,283,13,1,1,9,5,112,...,1,,1,,,,1,1,1,25981.966732


In [98]:
Person_Level_Data=Level2[['HH_ID','Person Srl No.',
       'Relation to head (code)', 'Gender', 'Age(in years)',
       'Marital Status (code)', 'Highest educational level attained (code)',
       'Total year of education completed',
       'Whether used internet from any location during last 30 days',
       'No. of days stayed away from home during last 30 days',
       'No. of meals usually taken in a day',
       'No. of meals taken during last 30 days from school, balwadi etc.',
       'No. of meals taken during last 30 days from employer as\r\r\nperquisites or part of wage',
       'No. of meals taken during last 30 days  others',
       'No. of meals taken during last 30 days on payment',
       'No. of meals taken during last 30 days at home']]

In [146]:
Person_Level_Data.columns=['HH_ID','Person Srl No.',
       'Relation to head (code)', 'Gender', 'Age(in years)',
       'Marital Status (code)', 'Highest educational level attained (code)',
       'Total year of education completed',
       'Whether used internet from any location during last 30 days',
       'No. of days stayed away from home during last 30 days',
       'No. of meals usually taken in a day',
       'No. of meals taken during last 30 days from school, balwadi etc.',
       'No. of meals taken during last 30 days from employer as perquisites or part of wage',
       'No. of meals taken during last 30 days  others',
       'No. of meals taken during last 30 days on payment',
       'No. of meals taken during last 30 days at home']

In [156]:
from sklearn.model_selection import train_test_split 
HH_Level_train_data, HH_Level_test_data = train_test_split(HH_Level_Data, test_size=0.2, random_state=42)

In [162]:
HH_Level_train_data.to_parquet(r"C:\Users\DIID\Downloads\HCES\HH_Level_train_data.parquet", index=False)
HH_Level_test_data.to_parquet(r"C:\Users\DIID\Downloads\HCES\HH_Level_test_data.parquet", index=False)

In [158]:
train_HH_ID= HH_Level_train_data['HH_ID'].tolist()
test_HH_ID= HH_Level_test_data['HH_ID'].tolist()

In [160]:
Person_Level_Train_Data = Person_Level_Data[Person_Level_Data['HH_ID'].isin(train_HH_ID)]
Person_Level_Test_Data = Person_Level_Data[Person_Level_Data['HH_ID'].isin(test_HH_ID)]

In [164]:
Person_Level_Train_Data.to_parquet(r"C:\Users\DIID\Downloads\HCES\Person_Level_Train_Data.parquet", index=False)
Person_Level_Test_Data.to_parquet(r"C:\Users\DIID\Downloads\HCES\Person_Level_Test_Data.parquet", index=False)

In [188]:
HH_Level_test_data.shape

(52350, 35)